In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('data/morti.carcere.xls', header=3)
df

,Cognome,Nome,Età,Data,Causa,Istituto,Note
0,Scarlata,Michele,66 anni,2024-01-28,Suicidio,Imperia,NaN
1,Caccamo,Francesco,59 anni,2024-01-26,Malattia,Santa Maria C.V. (Ce),Carcere militare
2,Lucera,Ivano,35 anni,2024-01-25,Suicidio,Foggia,NaN
3,Elsayed,Ahmed Adel,34 anni,2024-01-25,Suicidio,Rossano Calabro (Cs),NaN
4,Tunisino,(nome sconosciuto),36 anni,2024-01-25,Da accertare,Ancona Montacuto,NaN
...,...,...,...,...,...,...,...
2169,Martera,Luigi,36 anni,2002-02-02,Malattia,Teramo,NaN
2170,Montella,Raffaele,40 anni,2002-01-30,Suicidio,Napoli Poggioreale,NaN
2171,Italiano,(nome sconosciuto),27 anni,2002-01-20,Overdose,Is Arenas (CA),NaN
2172,Serbo,(nome sconosciuto),24 anni,2002-01-17,Suicidio,Sassari,NaN


In [3]:
df = df[['Istituto', 'Causa', 'Data']]
df = df[df['Data'] >= '2018-01-01']
df


,Istituto,Causa,Data
0,Imperia,Suicidio,2024-01-28
1,Santa Maria C.V. (Ce),Malattia,2024-01-26
2,Foggia,Suicidio,2024-01-25
3,Rossano Calabro (Cs),Suicidio,2024-01-25
4,Ancona Montacuto,Da accertare,2024-01-25
...,...,...,...
724,Civitavecchia (Rm),Suicidio,2018-01-15
725,Cagliari,Suicidio,2018-01-14
726,Viterbo,Da accertare,2018-01-09
727,Parma,Malattia,2018-01-05


In [4]:
# Load the mapping of Istituto to Regione
istituti_regioni = pd.read_csv('data/istituti-regioni.csv')

# Merge the dataframe with the mapping to add the Regione column
df = pd.merge(df, istituti_regioni, on='Istituto', how='left')
df


,Istituto,Causa,Data,Region
0,Imperia,Suicidio,2024-01-28,Liguria
1,Santa Maria C.V. (Ce),Malattia,2024-01-26,Campania
2,Foggia,Suicidio,2024-01-25,Puglia
3,Rossano Calabro (Cs),Suicidio,2024-01-25,Calabria
4,Ancona Montacuto,Da accertare,2024-01-25,Marche
...,...,...,...,...
724,Civitavecchia (Rm),Suicidio,2018-01-15,Lazio
725,Cagliari,Suicidio,2018-01-14,Sardegna
726,Viterbo,Da accertare,2018-01-09,Lazio
727,Parma,Malattia,2018-01-05,Emilia Romagna


In [5]:
# Convert 'Data' to datetime format to extract year and month
df['Data'] = pd.to_datetime(df['Data'])

# Create a 'Month-Year' column for grouping
df['Month-Year'] = df['Data'].dt.to_period('M')

# Group by 'Region' and 'Month-Year', then count the number of deaths
deaths_per_region_month = df.groupby(['Region', 'Month-Year']).size().reset_index(name='Number of Deaths')

# Display the final dataframe
deaths_per_region_month


,Region,Month-Year,Number of Deaths
0,Abruzzo,2018-03,2
1,Abruzzo,2018-04,1
2,Abruzzo,2018-05,1
3,Abruzzo,2019-08,1
4,Abruzzo,2020-07,1
...,...,...,...
497,Veneto,2023-07,1
498,Veneto,2023-08,1
499,Veneto,2023-11,2
500,Veneto,2023-12,3


In [6]:
deaths_per_region_month.to_csv("data/deaths_regions.csv", index=False)

In [7]:
# Create a new dataframe for total deaths per month regardless of the Region
total_deaths_per_month = deaths_per_region_month.groupby('Month-Year')['Number of Deaths'].sum().reset_index(name='Total Deaths')

# Display the new dataframe
total_deaths_per_month


,Month-Year,Total Deaths
0,2018-01,9
1,2018-02,6
2,2018-03,9
3,2018-04,7
4,2018-05,11
...,...,...
68,2023-09,12
69,2023-10,8
70,2023-11,9
71,2023-12,8


In [8]:
total_deaths_per_month.to_csv("data/deaths_totals.csv", index=False)

In [9]:
deaths_per_cause_month = df.groupby(['Cause', 'Month-Year']).size().reset_index(name='Number of Deaths')
deaths_per_cause_month


,Istituto,Causa,Data,Region,Month-Year
0,Imperia,Suicidio,2024-01-28,Liguria,2024-01
1,Santa Maria C.V. (Ce),Malattia,2024-01-26,Campania,2024-01
2,Foggia,Suicidio,2024-01-25,Puglia,2024-01
3,Rossano Calabro (Cs),Suicidio,2024-01-25,Calabria,2024-01
4,Ancona Montacuto,Da accertare,2024-01-25,Marche,2024-01
...,...,...,...,...,...
724,Civitavecchia (Rm),Suicidio,2018-01-15,Lazio,2018-01
725,Cagliari,Suicidio,2018-01-14,Sardegna,2018-01
726,Viterbo,Da accertare,2018-01-09,Lazio,2018-01
727,Parma,Malattia,2018-01-05,Emilia Romagna,2018-01


In [11]:
deaths_per_cause_month = df.groupby(['Causa', 'Month-Year']).size().reset_index(name='Number of Deaths')
deaths_per_cause_month

,Causa,Month-Year,Number of Deaths
0,Da Accertare,2018-04,1
1,Da accertare,2018-01,2
2,Da accertare,2018-02,1
3,Da accertare,2018-03,4
4,Da accertare,2018-05,3
...,...,...,...
204,Suicidio,2023-09,6
205,Suicidio,2023-10,4
206,Suicidio,2023-11,6
207,Suicidio,2023-12,5


In [12]:
deaths_per_cause_month.to_csv("data/deaths_causes.csv", index=False)